# Imports

In [114]:
from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

In [115]:
import os
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from time import strftime
from PIL import Image # Pillow lib for image processing


# Constants

In [116]:
X_TRAIN_PATH = 'MINIST\digit_xtrain.csv'
X_TEST_PATH = 'MINIST\digit_xtest.csv'
Y_TRAIN_PATH = 'MINIST\digit_ytrain.csv'
Y_TEST_PATH = 'MINIST\digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10 #no of digits to be determined i.e. 0-9(10 digits in total).
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1    #Black and white image so there will be a single channel whose value ranges from 0 to 255 meaning 0 as white and 255 as dark.
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT*CHANNELS # input to the first NN model.


# Get the Data

In [117]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
y_test= np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Explore

In [118]:
y_train_all.shape

(60000,)

In [119]:
x_train_all.shape

(60000, 784)

In [120]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [121]:
x_test.shape

(10000, 784)

In [122]:
y_test.shape

(10000,)

In [123]:
y_train_all[:5] # first five datas  of y_train_all

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [124]:
# Re-scale
# Black_white images has every pixel value ranging from 1 to 255 so dividing that value by 255.0 makes all the values ranging between 0 and 1.
x_train_all, x_test = x_train_all/255.0, x_test/255.0

#### Convert target values to one-hot encoding

In [125]:
values = y_train_all[:5]
np.eye(10)[values] #Return a 2-D array with ones on the diagonal and zeros elsewhere.

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [126]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [127]:
y_train_all.shape

(60000, 10)

In [128]:
y_test = np.eye(NR_CLASSES)[y_test]

In [129]:
y_test.shape

(10000, 10)

#### Create validation dataset from training data

In [130]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all  [:VALIDATION_SIZE]

x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [131]:
x_train.shape

(50000, 784)

In [132]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [133]:
X = tf.compat.v1.placeholder(tf.float32, shape=[None,TOTAL_INPUTS],name='X') # None means the inputs can vary in size.
# X = tf.keras.Input(shape=(TOTAL_INPUTS,))
y =tf.compat.v1.placeholder(tf.float32, shape=[None,NR_CLASSES],name='labels')  # same functionality as below.
# y= tf.keras.Input(shape=NR_CLASSES,)

# Neural Network Architecture
#### Hyperparameters

In [134]:
nr_epochs = 50
learning_rate = 1e-3 # 0.001 is also the same number

n_hidden1 = 512
n_hidden2 = 64

In [135]:
# Function to setup a layer
def setup_layer(input, weight_dim, bias_dim,name):
    with tf.name_scope(name):
        initial_w = tf.random.truncated_normal(shape=weight_dim,stddev=0.1,seed=42)
        w = tf.Variable(initial_value=initial_w,name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b,name='B')

        layer_in = tf.matmul(input, w) + b

        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.compat.v1.summary.histogram('weights',w)
        tf.compat.v1.summary.histogram('biases',b)

        return layer_out

In [136]:

# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[
#     n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], bias_dim=[
#     n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[
#     NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [137]:

layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[
    n_hidden1], name='layer_1')

layer_drop = tf.compat.v1.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], bias_dim=[
    n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[
    NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

# Tensorboard Setup

In [138]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH,folder_name)

try: 
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else: 
    print('Successfully created directories!')

Successfully created directories!


# Loss, Optimization & Metrics

In [139]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))

In [140]:
with tf.name_scope('optimizer'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

In [141]:
# Accuracy Metric

with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output,axis=1),tf.argmax(y,axis=1)) # checks if the predicted value and the actual value are equal.
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32)) # calculates the mean of all the correct predictions.

In [142]:
with tf.name_scope('performance'):
    tf.compat.v1.summary.scalar('accuracy',accuracy)
    tf.compat.v1.summary.scalar('cost',loss)

#### Check Input Images in Tensorboard

In [143]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X,[-1,IMAGE_WIDTH,IMAGE_HEIGHT,CHANNELS])
    tf.compat.v1.summary.image('input_image',x_image,max_outputs=4)

# Run Session

In [144]:
sess = tf.compat.v1.Session()

##### setup Filewriter and Megre Summaries

In [145]:
merged_summary = tf.compat.v1.summary.merge_all()

train_writer = tf.compat.v1.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.compat.v1.summary.FileWriter(directory+'/validation')
validation_writer.add_graph(sess.graph)

In [146]:
# initialise all the variables
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

# Batching the data

In [147]:
size_of_batch = 1000

In [148]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [149]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch
    return data[start:end], labels[start:end]

# Training Loop

In [150]:
for epoch in range(nr_epochs):

    #  ============= Training Dataset =============
    for i in range(nr_iterations):

        batch_x, batch_y = next_batch(
            batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X: batch_x, y: batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)

    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)

    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch}\t|Training Accuracy = {batch_accuracy}')


    #  ============= Validation =============
    summary = sess.run(fetches=merged_summary, feed_dict={X: x_val, y: y_val})
    validation_writer.add_summary(summary, epoch)
print('Done Training!')

Epoch 0	|Training Accuracy = 0.8629999756813049
Epoch 1	|Training Accuracy = 0.8809999823570251
Epoch 2	|Training Accuracy = 0.9179999828338623
Epoch 3	|Training Accuracy = 0.9620000123977661
Epoch 4	|Training Accuracy = 0.9739999771118164
Epoch 5	|Training Accuracy = 0.9750000238418579
Epoch 6	|Training Accuracy = 0.9769999980926514
Epoch 7	|Training Accuracy = 0.9800000190734863
Epoch 8	|Training Accuracy = 0.9800000190734863
Epoch 9	|Training Accuracy = 0.9800000190734863
Epoch 10	|Training Accuracy = 0.9800000190734863
Epoch 11	|Training Accuracy = 0.9850000143051147
Epoch 12	|Training Accuracy = 0.9860000014305115
Epoch 13	|Training Accuracy = 0.984000027179718
Epoch 14	|Training Accuracy = 0.9879999756813049
Epoch 15	|Training Accuracy = 0.9890000224113464
Epoch 16	|Training Accuracy = 0.9890000224113464
Epoch 17	|Training Accuracy = 0.984000027179718
Epoch 18	|Training Accuracy = 0.9900000095367432
Epoch 19	|Training Accuracy = 0.9900000095367432
Epoch 20	|Training Accuracy = 0.

# Making a Prediction

In [151]:
img = Image.open('MINIST/test_img.png')
img

In [152]:
bw = img.convert(mode='L') # convert to grayscale 


In [153]:
img_array = np.invert(bw) # invert the image

In [154]:
img_array.shape

(28, 28)

In [155]:
test_img = img_array.ravel()

In [156]:
test_img.shape

(784,)

In [157]:
prediction = sess.run(fetches=tf.argmax(output,axis=1),feed_dict={X:[test_img]})

In [158]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing & Evaluation

In [159]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test,y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.75%


# Reset for the Next Run

In [160]:
train_writer.close
validation_writer.close
sess.close
tf.compat.v1.reset_default_graph()

In [161]:
# Code to run in anaconda prompt to run tensorboard
# tensorboard --logdir=logs/ --port=6006 --host=localhost
# So in this case the code looks like this:
# tensorboard --logdir=learning/digits_recognition/tensorboard_mnist_digit_logs --port=6006 --host=localhost --bind_all serve


# Initial code of setting up layers one by one without using the function.

In [162]:
# # Setting up 1st hidden layer
# with tf.name_scope('First_hidden_layer'):
#     # Initializing weight matrix
#     initial_w1 = tf.random.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1],stddev=0.1,seed=42)
#     w1 = tf.Variable(initial_value=initial_w1,name='w1') #The variable w1 will be updated during the training process as the neural network learns from the data.

#     # Initializing a bias vector for a neural network layer 
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1,name='b1')

#     layer1_in = tf.matmul(X, w1)+b1

#     layer1_out = tf.nn.relu(layer1_in)

In [163]:
# # Setting up 2nd hidden layer
# with tf.name_scope('2nd_hidden_layer'):
#     initial_w2 = tf.random.truncated_normal(shape=[n_hidden1, n_hidden2],stddev=0.1,seed=42)
#     w2 = tf.Variable(initial_value=initial_w2,name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2,name='b2')

#     layer2_in = tf.matmul(layer1_out, w2)+b2

#     layer2_out = tf.nn.relu(layer2_in)

In [164]:
# # Setting up final hidden layer
# with tf.name_scope('output_layer'):
#     initial_w3 = tf.random.truncated_normal(shape=[n_hidden2, NR_CLASSES],stddev=0.1,seed=42)
#     w3 = tf.Variable(initial_value=initial_w3,name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3,name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3

#     output = tf.nn.softmax(layer3_in)

In [165]:
# b3.eval(sess)